**Step 1: Set up and Preprocessing**


In Colab, you need to turn text into numbers that the LSTM can understand.

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention, Concatenate, Bidirectional, Dropout


In [24]:
# --- 1. CONFIGURATION & PREPROCESSING ---
MAX_LEN = 25
EMBED_DIM = 256
LATENT_DIM = 512

def normalize_text(text):
    text = str(text).lower().strip()
    # Reduce repeated characters (heyyyy -> heyy)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    # Space out punctuation
    text = re.sub(r"([?.!,])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


In [25]:
# Load and prepare data
df = pd.read_csv('style_transfer_dataset (1).csv')
df['normal_text'] = df['normal_text'].apply(normalize_text)
# Add start/end tokens to target
df['professional_text'] = df['professional_text'].apply(lambda x: f'sos {normalize_text(x)} eos')


In [26]:
df.shape

(1000, 2)

In [27]:
df.sample(5)

,normal_text,professional_text
913,could you possibly look into this ?,sos your prompt attention to investigate this ...
184,could you possibly send the file ?,sos please proceed to forward the file . eos
796,would you handle this,sos your prompt attention to manage this matte...
887,i need you to review the document,sos please proceed to review the document . eos
380,would you send the file,sos i would appreciate it if you could forward...


In [28]:
# Tokenization
def build_tokenizer(texts):
    # We keep ! and ? because they matter for style
    tokenizer = Tokenizer(filters='"#$%&()*+,-/:;=@[\\]^_`{|}~\t\n', oov_token='<unk>')
    tokenizer.fit_on_texts(texts)
    return tokenizer

input_tokenizer = build_tokenizer(df['normal_text'])
target_tokenizer = build_tokenizer(df['professional_text'])

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

**🏗️ Step 2: The Encoder-Decoder with Attention**


We will build this using the TensorFlow Functional API. It consists of an Encoder (to understand the normal text) and a Decoder (to write the formal version).

In [29]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention, Concatenate

# Hyperparameters
latent_dim = 256
embed_dim = 128
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1



In [30]:
# Prepare sequences
encoder_input_data = pad_sequences(input_tokenizer.texts_to_sequences(df['normal_text']), maxlen=MAX_LEN, padding='post')
decoder_full_data = pad_sequences(target_tokenizer.texts_to_sequences(df['professional_text']), maxlen=MAX_LEN+1, padding='post')

# Decoder Input: sos -> word1 -> word2
# Decoder Target: word1 -> word2 -> eos
decoder_input_data = decoder_full_data[:, :-1]
decoder_target_data = decoder_full_data[:, 1:]

In [31]:
# --- 2. MODEL ARCHITECTURE ---
# Encoder
encoder_inputs = Input(shape=(MAX_LEN,))
enc_emb = Embedding(input_vocab_size, EMBED_DIM)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(LATENT_DIM, return_sequences=True, return_state=True, dropout=0.3))
encoder_outputs, fh, fc, bh, bc = encoder_lstm(enc_emb)
state_h = Concatenate()([fh, bh])
state_c = Concatenate()([fc, bc])

# Decoder
decoder_inputs = Input(shape=(MAX_LEN,))
dec_emb_layer = Embedding(target_vocab_size, EMBED_DIM)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(LATENT_DIM * 2, return_sequences=True, return_state=True, dropout=0.3)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Attention
attention_layer = Attention()
context_vector = attention_layer([decoder_outputs, encoder_outputs])
decoder_combined = Concatenate(axis=-1)([decoder_outputs, context_vector])

# Output
decoder_dense = Dense(target_vocab_size, activation='softmax')
outputs = decoder_dense(decoder_combined)


In [32]:
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy')

**⏳ Step 3: Training the Model**


In Colab, you can now run the training loop. Since we have a small dataset, we will train for more epochs (like 50-100).

In [33]:
# --- 3. TRAINING ---
print("Starting training... aimed at generalizing, not just memorizing.")
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=[callback]
)


Starting training... aimed at generalizing, not just memorizing.
Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: 3.9045 - val_loss: 1.5214
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.5021 - val_loss: 1.3545
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.3457 - val_loss: 1.2587
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.2206 - val_loss: 1.1551
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.1065 - val_loss: 1.0436
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.9993 - val_loss: 0.9674
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.8953 - val_loss: 0.8756
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.8059 - val_loss: 0.7755
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.6911 - val_loss: 0.7124
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.6091 - val_loss: 0.6562
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.5748 - val_

In [34]:
# --- 4. ROBUST INFERENCE (TRANSLATION) ---
def translate_to_formal(sentence):
    clean_input = normalize_text(sentence)
    input_seq = input_tokenizer.texts_to_sequences([clean_input])
    input_padded = pad_sequences(input_seq, maxlen=MAX_LEN, padding='post')

    # Initialize decoder input with 'sos'
    target_seq = np.zeros((1, MAX_LEN))
    target_seq[0, 0] = target_tokenizer.word_index.get('sos', 1)

    translated_words = []

    for i in range(MAX_LEN - 1):
        output = model.predict([input_padded, target_seq], verbose=0)

        # Get the word for the current index i
        idx = np.argmax(output[0, i, :])
        word = target_tokenizer.index_word.get(idx, '')

        if word == 'eos' or word == '' or idx == 0:
            break

        translated_words.append(word)
        target_seq[0, i+1] = idx # Feed the predicted word back for next step

    return " ".join(translated_words)


# Task
Adjust hyperparameters, rebuild, and retrain the style transfer model, then evaluate its performance on unseen data to determine if reducing model capacity mitigates overfitting and leads to more accurate and formal translations. Provide an assessment of the model's performance and suggest any further steps.

## Adjust Hyperparameters

### Subtask:
Reduce the model's capacity by adjusting the `LATENT_DIM` and `EMBED_DIM` hyperparameters.


**Reasoning**:
The subtask requires adjusting the `EMBED_DIM` and `LATENT_DIM` hyperparameters. I will update these values in a new code block.



In [37]:
EMBED_DIM = 128
LATENT_DIM = 256

print(f"Updated EMBED_DIM: {EMBED_DIM}")
print(f"Updated LATENT_DIM: {LATENT_DIM}")

Updated EMBED_DIM: 128
Updated LATENT_DIM: 256


**Reasoning**:
The previous step updated the `EMBED_DIM` and `LATENT_DIM` hyperparameters. Now, the model architecture needs to be re-defined and re-compiled using these new values to reflect the reduced capacity.



In [38]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention, Concatenate, Bidirectional, Dropout

# --- 2. MODEL ARCHITECTURE ---
# Encoder
encoder_inputs = Input(shape=(MAX_LEN,))
enc_emb = Embedding(input_vocab_size, EMBED_DIM)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(LATENT_DIM, return_sequences=True, return_state=True, dropout=0.3))
encoder_outputs, fh, fc, bh, bc = encoder_lstm(enc_emb)
state_h = Concatenate()([fh, bh])
state_c = Concatenate()([fc, bc])

# Decoder
decoder_inputs = Input(shape=(MAX_LEN,))
dec_emb_layer = Embedding(target_vocab_size, EMBED_DIM)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(LATENT_DIM * 2, return_sequences=True, return_state=True, dropout=0.3)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Attention
attention_layer = Attention()
context_vector = attention_layer([decoder_outputs, encoder_outputs])
decoder_combined = Concatenate(axis=-1)([decoder_outputs, context_vector])

# Output
decoder_dense = Dense(target_vocab_size, activation='softmax')
outputs = decoder_dense(decoder_combined)

model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy')


## Rebuild and Retrain Model

### Subtask:
Retrain the model with the adjusted hyperparameters.


**Reasoning**:
The previous steps updated the hyperparameters and rebuilt the model. Now, I need to retrain the model using the adjusted hyperparameters, following the instructions for batch size, epochs, validation split, and callbacks.



In [39]:
print("Starting training with adjusted hyperparameters...")
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=60,
    validation_split=0.2,
    callbacks=[callback]
)

Starting training with adjusted hyperparameters...
Epoch 1/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 4.3937 - val_loss: 1.5796
Epoch 2/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.5274 - val_loss: 1.4070
Epoch 3/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 1.3723 - val_loss: 1.3099
Epoch 4/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.2753 - val_loss: 1.2148
Epoch 5/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.1838 - val_loss: 1.1392
Epoch 6/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.0946 - val_loss: 1.0603
Epoch 7/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.0180 - val_loss: 0.9820
Epoch 8/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.9325 - val_loss: 0.9117
Epoch 9/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.8617 - val_loss: 0.8499
Epoch 10/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.7824 - val_loss: 0.7933
Epoch 11/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.7009 - val_loss: 0.7538
Epoch 12/60


**Reasoning**:
The model has been retrained with adjusted hyperparameters. Now, I need to evaluate its performance on unseen data and provide an assessment of its performance, comparing it to the previous training results.



In [40]:
print("\n" + "="*30)
test_inputs = [
    "hey how are you??",
    "send me the file ASAP",
    "i'm so tired of this project",
    "let's talk tomorrow"
]

for t in test_inputs:
    result = translate_to_formal(t)
    print(f"Casual: {t}")
    print(f"Formal: {result}")
    print("-" * 10)


Casual: hey how are you??
Formal: i will respond the delayed of this
----------
Casual: send me the file ASAP
Formal: i would appreciate it if you could review the document .
----------
Casual: i'm so tired of this project
Formal: i am concerns regarding this approach
----------
Casual: let's talk tomorrow
Formal: this is executed well
----------
